In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import itertools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import os

# 📁 Save directory for plots
plot_dir = r"C:\Users\vishn\OneDrive\Documents\Machine Learning\GridSearch"
os.makedirs(plot_dir, exist_ok=True)

# Force CPU usage
device = torch.device("cpu")
torch.cuda.is_available = lambda: False

# Set seed for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)

# Define the ANN model
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# Load and prepare dataset
df = pd.read_csv(r"C:\Users\vishn\OneDrive\Documents\Machine Learning\Vishnu_phd.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["FoS"].values  # 🔁 Change to df["SeismicFoS"].values for seismic model

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Grid search space
h1_vals = [32, 64, 96]
h2_vals = [32, 64, 96]
h3_vals = [32, 64]
lr_vals = [0.001, 0.003, 0.005]
param_grid = list(itertools.product(h1_vals, h2_vals, h3_vals, lr_vals))

best_r2 = -np.inf
best_model = None
best_params = {}

# Grid search
for h1, h2, h3, lr in param_grid:
    set_seed()
    model = ANN(X_train.shape[1], h1, h2, h3).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        train_preds = model(X_train_tensor).cpu().numpy()
        test_preds = model(X_test_tensor).cpu().numpy()

    r2_train = r2_score(y_train, train_preds)
    r2_test = r2_score(y_test, test_preds)
    rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
    rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
    mae_train = mean_absolute_error(y_train, train_preds)
    mae_test = mean_absolute_error(y_test, test_preds)

    if r2_test > best_r2:
        best_r2 = r2_test
        best_model = model
        best_params = {
            "h1": h1, "h2": h2, "h3": h3, "lr": lr,
            "r2_train": r2_train, "r2_test": r2_test,
            "rmse_train": rmse_train, "rmse_test": rmse_test,
            "mae_train": mae_train, "mae_test": mae_test,
            "train_preds": train_preds,
            "test_preds": test_preds,
        }

# Print best result
print("\n📊 Best Grid Search Result for FoS:")
print(f"Architecture: ({best_params['h1']}, {best_params['h2']}, {best_params['h3']}) | Learning Rate: {best_params['lr']}")
print(f"✅ R² (Train): {best_params['r2_train']:.6f}")
print(f"✅ R² (Test):  {best_params['r2_test']:.6f}")
print(f"📉 RMSE (Train): {best_params['rmse_train']:.6f}")
print(f"📉 RMSE (Test):  {best_params['rmse_test']:.6f}")
print(f"📉 MAE (Train): {best_params['mae_train']:.6f}")
print(f"📉 MAE (Test):  {best_params['mae_test']:.6f}")

# 📈 Plot: Actual vs Predicted
fig1 = make_subplots(rows=1, cols=2, subplot_titles=("Train Predictions", "Test Predictions"))
fig1.add_trace(go.Scatter(y=y_train, mode="lines", name="Actual (Train)", line=dict(dash='dot')), row=1, col=1)
fig1.add_trace(go.Scatter(y=best_params['train_preds'].flatten(), mode="lines", name="Predicted (Train)"), row=1, col=1)
fig1.add_trace(go.Scatter(y=y_test, mode="lines", name="Actual (Test)", line=dict(dash='dot')), row=1, col=2)
fig1.add_trace(go.Scatter(y=best_params['test_preds'].flatten(), mode="lines", name="Predicted (Test)"), row=1, col=2)
fig1.update_layout(title_text="Actual vs Predicted - Train and Test Sets", height=500, width=1000)
fig1.write_html(os.path.join(plot_dir, "actual_vs_predicted.html"))
print("✅ Saved: actual_vs_predicted.html")

# 📈 Plot: Residuals
residual_train = y_train - best_params['train_preds'].flatten()
residual_test = y_test - best_params['test_preds'].flatten()
fig2 = make_subplots(rows=1, cols=2, subplot_titles=("Train Residuals", "Test Residuals"))
fig2.add_trace(go.Scatter(y=residual_train, mode='markers', name='Residuals (Train)'), row=1, col=1)
fig2.add_trace(go.Scatter(y=residual_test, mode='markers', name='Residuals (Test)'), row=1, col=2)
fig2.update_layout(title_text="Residuals Plot", height=500, width=1000)
fig2.write_html(os.path.join(plot_dir, "residuals.html"))
print("✅ Saved: residuals.html")

# 📊 Plot: Metric Comparison
fig3 = go.Figure(data=[
    go.Bar(name='Train', x=['R²', 'RMSE', 'MAE'],
           y=[best_params['r2_train'], best_params['rmse_train'], best_params['mae_train']]),
    go.Bar(name='Test', x=['R²', 'RMSE', 'MAE'],
           y=[best_params['r2_test'], best_params['rmse_test'], best_params['mae_test']])
])
fig3.update_layout(barmode='group', title="Performance Metrics Comparison", height=400, width=800)
fig3.write_html(os.path.join(plot_dir, "performance_metrics_comparison.html"))
print("✅ Saved: performance_metrics_comparison.html")



📊 Best Grid Search Result for FoS:
Architecture: (32, 32, 64) | Learning Rate: 0.001
✅ R² (Train): 0.966605
✅ R² (Test):  0.917855
📉 RMSE (Train): 0.210712
📉 RMSE (Test):  0.364616
📉 MAE (Train): 0.120773
📉 MAE (Test):  0.230364
✅ Saved: actual_vs_predicted.html
✅ Saved: residuals.html
✅ Saved: performance_metrics_comparison.html


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import itertools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import os

# 📁 Save directory for plots
plot_dir = r"C:\Users\vishn\OneDrive\Documents\Machine Learning\GridSearch\Seismic"
os.makedirs(plot_dir, exist_ok=True)

# Force CPU usage
device = torch.device("cpu")
torch.cuda.is_available = lambda: False

# Set seed for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)

# Define the ANN model
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# Load and prepare dataset
df = pd.read_csv(r"C:\Users\vishn\OneDrive\Documents\Machine Learning\Vishnu_phd.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["SeismicFoS"].values  # 🔁 Changed to SeismicFoS

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Grid search space
h1_vals = [32, 64, 96]
h2_vals = [32, 64, 96]
h3_vals = [32, 64]
lr_vals = [0.001, 0.003, 0.005]
param_grid = list(itertools.product(h1_vals, h2_vals, h3_vals, lr_vals))

best_r2 = -np.inf
best_model = None
best_params = {}

# Grid search
for h1, h2, h3, lr in param_grid:
    set_seed()
    model = ANN(X_train.shape[1], h1, h2, h3).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        train_preds = model(X_train_tensor).cpu().numpy()
        test_preds = model(X_test_tensor).cpu().numpy()

    r2_train = r2_score(y_train, train_preds)
    r2_test = r2_score(y_test, test_preds)
    rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
    rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
    mae_train = mean_absolute_error(y_train, train_preds)
    mae_test = mean_absolute_error(y_test, test_preds)

    if r2_test > best_r2:
        best_r2 = r2_test
        best_model = model
        best_params = {
            "h1": h1, "h2": h2, "h3": h3, "lr": lr,
            "r2_train": r2_train, "r2_test": r2_test,
            "rmse_train": rmse_train, "rmse_test": rmse_test,
            "mae_train": mae_train, "mae_test": mae_test,
            "train_preds": train_preds,
            "test_preds": test_preds,
        }

# Print best result
print("\n📊 Best Grid Search Result for SeismicFoS:")
print(f"Architecture: ({best_params['h1']}, {best_params['h2']}, {best_params['h3']}) | Learning Rate: {best_params['lr']}")
print(f"✅ R² (Train): {best_params['r2_train']:.6f}")
print(f"✅ R² (Test):  {best_params['r2_test']:.6f}")
print(f"📉 RMSE (Train): {best_params['rmse_train']:.6f}")
print(f"📉 RMSE (Test):  {best_params['rmse_test']:.6f}")
print(f"📉 MAE (Train): {best_params['mae_train']:.6f}")
print(f"📉 MAE (Test):  {best_params['mae_test']:.6f}")

# 📈 Plot: Actual vs Predicted
fig1 = make_subplots(rows=1, cols=2, subplot_titles=("Train Predictions", "Test Predictions"))
fig1.add_trace(go.Scatter(y=y_train, mode="lines", name="Actual (Train)", line=dict(dash='dot')), row=1, col=1)
fig1.add_trace(go.Scatter(y=best_params['train_preds'].flatten(), mode="lines", name="Predicted (Train)"), row=1, col=1)
fig1.add_trace(go.Scatter(y=y_test, mode="lines", name="Actual (Test)", line=dict(dash='dot')), row=1, col=2)
fig1.add_trace(go.Scatter(y=best_params['test_preds'].flatten(), mode="lines", name="Predicted (Test)"), row=1, col=2)
fig1.update_layout(title_text="Actual vs Predicted - Train and Test Sets (SeismicFoS)", height=500, width=1000)
fig1.write_html(os.path.join(plot_dir, "seismicfos_actual_vs_predicted.html"))
print("✅ Saved: seismicfos_actual_vs_predicted.html")

# 📈 Plot: Residuals
residual_train = y_train - best_params['train_preds'].flatten()
residual_test = y_test - best_params['test_preds'].flatten()
fig2 = make_subplots(rows=1, cols=2, subplot_titles=("Train Residuals", "Test Residuals"))
fig2.add_trace(go.Scatter(y=residual_train, mode='markers', name='Residuals (Train)'), row=1, col=1)
fig2.add_trace(go.Scatter(y=residual_test, mode='markers', name='Residuals (Test)'), row=1, col=2)
fig2.update_layout(title_text="Residuals Plot (SeismicFoS)", height=500, width=1000)
fig2.write_html(os.path.join(plot_dir, "seismicfos_residuals.html"))
print("✅ Saved: seismicfos_residuals.html")

# 📊 Plot: Metric Comparison
fig3 = go.Figure(data=[
    go.Bar(name='Train', x=['R²', 'RMSE', 'MAE'],
           y=[best_params['r2_train'], best_params['rmse_train'], best_params['mae_train']]),
    go.Bar(name='Test', x=['R²', 'RMSE', 'MAE'],
           y=[best_params['r2_test'], best_params['rmse_test'], best_params['mae_test']])
])
fig3.update_layout(barmode='group', title="Performance Metrics Comparison (SeismicFoS)", height=400, width=800)
fig3.write_html(os.path.join(plot_dir, "seismicfos_performance_metrics_comparison.html"))
print("✅ Saved: seismicfos_performance_metrics_comparison.html")



📊 Best Grid Search Result for SeismicFoS:
Architecture: (96, 32, 64) | Learning Rate: 0.003
✅ R² (Train): 0.991743
✅ R² (Test):  0.863500
📉 RMSE (Train): 0.079358
📉 RMSE (Test):  0.346894
📉 MAE (Train): 0.053264
📉 MAE (Test):  0.231047
✅ Saved: seismicfos_actual_vs_predicted.html
✅ Saved: seismicfos_residuals.html
✅ Saved: seismicfos_performance_metrics_comparison.html
